In [0]:
import wikipedia
wikipedia.set_lang("ru")

from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, is_capitalized, dictionary, eq
import re
import pandas as pd
from tqdm import tqdm_notebook
from gensim import utils

import warnings
warnings.filterwarnings('ignore')

START = rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN')
)

START_S = or_(
    eq('такой'),
    eq('такие'),
)

KAK = eq('как')
INCLUDING = or_(
    or_(
        eq('в'),
        eq('том'),
        eq('числе'),
    ),
    eq('включающий'),
    or_(
        eq('включающий'),
        eq('в'),
        eq('себя'),
    ),
    or_(
        eq('включающие'),
        eq('в'),
        eq('себя'),
    ),
    eq('включающие'),
    eq('особенно'),

)

MID_S = or_(
    rule(
        or_(
            eq('такой'),
            eq('такие'),
        ),
        eq('как')
    )
)
ATAKJE = rule(
    eq(','),
    eq('а'),
    eq('также')
)

MID = or_(
    rule(
        eq('это')
    ),
    rule(
        eq('—')
    ),
    rule(
        eq('—'),
        eq('это')
    ),
    rule(
        eq('—'),
        not_(eq('км'))
    ),
    rule(
        or_(
            eq('и'),
            eq('или'),
        ),
        eq('другие')
    )
)

END = or_(
    rule(
        gram('NOUN'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable()
    )
)

Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)


IGNORE = rule(
    '(',
    not_(eq(')')).repeatable(),
    ')'
)

ITEM = rule(
    IGNORE.interpretation(
        Item.titles
    ),
    eq(',').optional() 
).repeatable().interpretation(
    Item
)

In [0]:
def get_hyperonyms(main_word):
    HYPONYM = eq(utils.deaccent(main_word))
    RULE = or_(
        rule(HYPONYM, ATAKJE, START, MID, END),
        rule(HYPONYM, MID, END),
        rule(START_S, END, KAK, HYPONYM),
        rule(END, INCLUDING, HYPONYM)
    )
    parser = Parser(RULE) 
    text = utils.deaccent(wikipedia.summary(main_word))
    print(text)
    text = re.sub(r'\(.+?\)', '', text)
    for idx, match in enumerate(parser.findall(text.lower())):
        k = [_.value for _ in match.tokens]
        print(k)

In [5]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/maira1207/NLP/master/lab2/Lab2.csv")
charge = df.loc[df['IN CHARGE NAME'] == "Abiyr Maira"]
charge.head(5)

,Заполнено,Unnamed: 1,IN CHARGE NUMBER,IN CHARGE NAME,HYPONYM,HYPERNYM,HYPERNYM.1,HYPERNYM.2,HYPERNYM.3,HYPERNYM.4,HYPERNYM.5,HYPERNYM.6,HYPERNYM.7,HYPERNYM.8,HYPERNYM.9,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,#ERROR!
1296,NaN,1297,55,Abiyr Maira,ТИГЕЛЬ,КУВШИН,СОСУД,ВМЕСТИЛИЩЕ,МЕСТО,ПРОСТРАНСТВО,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1297,NaN,1298,55,Abiyr Maira,ТИМИН,ОСНОВАНИЕ,НУКЛЕОТИД,ГРУППА,СОВОКУПНОСТЬ,СОЧЕТАНИЕ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1298,NaN,1299,55,Abiyr Maira,ТИПОЛОГИЗАЦИЯ,МЕТОД,ПУТЬ,МЕСТО,ИССЛЕДОВАНИЯ,ПОЗНАНИЕ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1299,NaN,1300,55,Abiyr Maira,ТИПОРАЗМЕР,СТАНДАРТ,ОБРАЗЕЦ,ИЗДЕЛИЕ,РАЗМЕР,ФОРМА,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1300,NaN,1301,55,Abiyr Maira,ТИХООКЕАНЕЦ,МОРЯК,РАБОТНИК,МАТРОС,ПРОФЕССИЯ,СПЕЦИАЛИСТ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
for index, row in charge.iterrows():
    try:
      print(row['HYPONYM'])
      get_hyperonyms(row['HYPONYM'])
      print("\n\n")
    except: 
        pass

ТИГЕЛЬ
Тигель (от нем. Tiegel — горшок) — это емкость для нагрева, высушивания, сжигания, обжига или плавления различных материалов. Тигли — это неотъемлемая часть металлургического и лабораторного оборудования при литье различных металлов, сплавов, и пр. Отличительнои особенностью тиглеи является применение для их конструкции огнеупорных материалов и высокоустоичивых к различным воздеиствиям металлов и сплавов.
Тигель имеет обычно коническую (усеченныи конус) или цилиндрическую форму. Разновидностью тиглеи являются также плавильные чашки, плавильные лодочки.



ТИМИН
Тимин (5-метилурацил) —  производное пиримидина, одно из пяти азотистых основании. Присутствует во всех живых организмах, где вместе с дезоксирибозои входит в состав нуклеозида тимидина, которыи может фосфорилироваться 1—3 остатками фосфорнои кислоты с образованием нуклеотидов тимидин моно-, ди- или трифосфорнои кислоты (ТМФ, ТДФ и ТТФ). Дезоксирибонуклеотиды тимина входят в состав ДНК, в РНК на его месте располагается ри